
start a bunch of front end services?


In [ ]:

const {spawn, spawnSync} = require("child_process");
const path = require('path')

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE

const COMMANDS = [
  ['node', '--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'start a bunch of llm rpc services'],
  ['node', '--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'start a bunch of discord services'],
  ['cloudflared', 'tunnel', 'run', '64d274ab-b0a4-4ef0-9636-e623dd02ac5c'],
  [path.join(HOMEPATH, 'stable-diffusion-webui', './webui.sh')],
  ['ollama', 'run', 'llama3.2-vision'],
  ['node', '--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'linkedin service'],
  //['node', '--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'github service'],
]

const ENVIRONMENTS = [
  {},
  {},
  {},
  {},
  {},
  {
    // for emailing
    ADMIN_USER: 'megamindbrian',
    ADMIN_PASS: JSON.parse(fs.readFileSync(path.join(HOMEPATH, '.credentials', 'google-secret.json')))
  },
  //{},
]

function launchChats(botIndex = 0) {

  if(parseInt(botIndex).toString() != botIndex.toString()) {
    return
  } else {
    botIndex = parseInt(botIndex)
  }

  if(typeof botIndex != 'number' && !botIndex) {
    return
  }

  for(let i = botIndex; i < ENVIRONMENTS.length; i++) {
    if(!ENVIRONMENTS[i] || !COMMANDS[i]) {
      continue
    }

    spawn(COMMANDS[i][0], COMMANDS[i].slice(1), {
      env: Object.assign({}, process.env, ENVIRONMENTS[i] ? ENVIRONMENTS[i] : {}),
      stdio: [0, 1, 2],
      //detached: true,
      //shell: true,
      cwd: ENVIRONMENTS[i].CWD || path.dirname(__dirname)
    })

    if(botIndex) {
      break
    }
  }
}

module.exports = launchChats

